# Spark Streaming com Twitter

In [142]:
!pip install requests_oauthlib

In [143]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime,strftime
import requests
import time
import string
import ast
import json

In [144]:
#Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [145]:

# @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_f0d6ce32_5e0f_4bc0_8812_229b8d429dbe',
  'project_id':'9a0cc60102244d368e96a83f25d4ca89',
  'region':'dallas',
  'user_id':'0caf8026c98a4342ac027a05416e6dee',
  'domain_id':'3be46074545f4c09b1f10df3ace95998',
  'domain_name':'1351407',
  'username':'member_327b95c3eecf105b8bdb0125b81968cfcc557dbd',
  'password':"""D[Cvr1bgf9DM^I{C""",
  'container':'CursoSpark',
  'tenantId':'undefined',
  'filename':'dataset_analise_sentimento.csv'
}


In [146]:
def set_hadoop_config_with_credentials(creds):  
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    # you can choose any name
    name = 'keystone'

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', creds['project_id'])
    hconf.set(prefix + '.username', creds['user_id'])
    hconf.set(prefix + '.password', creds['password'])
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

In [147]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '9a0cc60102244d368e96a83f25d4ca89')
    hconf.set(prefix + '.username', '0caf8026c98a4342ac027a05416e6dee')
    hconf.set(prefix + '.password', 'D[Cvr1bgf9DM^I{C')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name)

spark = SparkSession.builder.getOrCreate()

df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://CursoSpark.' + name + '/dataset_analise_sentimento.csv')


AttributeError: 'NoneType' object has no attribute 'hadoopConfiguration'

In [ ]:
set_hadoop_config_with_credentials(credentials_1) 
fileNameOut = 'swift://'+ credentials_1['container'] + '.keystone/dataset_analise_sentimento.csv' 
arquivo = sc.textFile(fileNameOut)

In [ ]:
#Frequencia de update em segundos
INTERVALO_BATCH = 5

In [ ]:
#Criando o Streaming Context
scc = StreamingContext(sc,INTERVALO_BATCH)

In [ ]:
arquivo.take(5)

In [ ]:
#Removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line : line != header)


In [ ]:
type(dataset)

In [ ]:
# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(line):
  row = line.split(',')
  sentimento = row[1]
  tweet = row[3].strip()
  translator = str.maketrans({key: None for key in string.punctuation})
#translator = re.compile('[%s]' % re.escape(string.punctuation))
#tweet = regex.sub('', tweet)
  tweet = tweet.translate(translator)
  tweet = tweet.split(' ')
  tweet_lower = []
  for word in tweet:
    tweet_lower.append(word.lower())
  return (tweet_lower, sentimento)

In [ ]:
# Aplcia a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
# Cria um objeto SentimentAnalyzer 
sentiment_analyzer = SentimentAnalyzer()

In [ ]:
#Certifique se de ter espaço em disco - Aproximadamente 5GB
nltk.download("stopwords")

In [ ]:
#Stopwords... palavras que nao devem ser processadas
stopwords_all = []

In [ ]:
for word in stopwords.words("english"):
    stopwords_all.append(word)
    stopwords_all.append(word + "_NEG")

In [ ]:
#Obtem 10000 tweets do dataset de treino e retorna as palavras que nao sao stopwords
dataset_treino_amostra = dataset_treino.take(10000)

all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
#Cria um unigram e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops,top_n=200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats,unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
#Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer=trainer,training_set=training_set)

In [ ]:
#Testa o classificador em algumas sentenças
test_sentence1 = [(["this","program","is","bad"],"")]
test_set = sentiment_analyzer.apply_features(test_sentence1)

In [ ]:
#Autenticacao do Twitter
consumer_key="bxAZviP8aBu45zmqzACuSdV5A"
consumer_secret="OqnhD4qAB1nsi6V2AgHZlinJYs3WupPMgfB8pixgWUB3DNfToX"
access_token="148351945-8wpWXpGzgY0SM5OHCEXQwlfSCrPg5kH5tnQEgiag"
access_token_secret="ITtY8GC5Bv63CxaobchBEHi5XHsERxaCOqAqH1NjGoxgi"

In [ ]:
#Especifica a URL termo de busca
search_term = "Dilma"
sample_url = "https://stream.twitter.com/1.1/statuses/sample.json"
filter_url = "https://stream.twitter.com/1.1/statuses/filter.json?track=" + search_term

In [ ]:
#Criando o objeto de autenticacao do twitter
auth = requests_oauthlib.OAuth1(consumer_key,consumer_secret,access_token,access_token_secret)

In [ ]:
#Configurando o Stream
rdd = scc.sparkContext.parallelize([0])
stream = scc.queueStream([],default=rdd)
type(stream)

In [ ]:
# Total de tweets por update
NUM_TWEETS = 500  

In [ ]:
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
  response = requests.get(filter_url, auth = auth, stream = True)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
  sentence = [(tweet, '')]
  test_set = sentiment_analyzer.apply_features(sentence)
  print(tweet, classifier.classify(test_set[0][0]))
  return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
  for line in rdd:
    tweet = line.strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
      tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [ ]:
# Cria uma lista vazia para os resultados
resultados = []

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
  global resultados
  pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
  counts = pairs.reduceByKey(add)
  output = []
  for count in counts.collect():
    output.append(count)
  result = [time.strftime("%I:%M:%S"), output]
  resultados.append(result)
  print(result)

In [ ]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
# Start streaming
scc.start()
# ssc.awaitTermination()

In [ ]:
#scc.stop()